In [1]:
import mne

data_path = '../data/001_D1.bdf'
bad_channels = ['Temp', 'Plet', 'Resp', 'Erg2', 'Erg1', 'GSR2', 'GSR1', 'EXG8', 'EXG7', 'EXG6', 'EXG5', 'EXG4', 'EXG3', 'EXG2', 'EXG1']
raw = mne.io.read_raw_bdf(data_path, preload=True)

raw.info['bads'] = bad_channels
raw.pick_types(eeg=True, meg=False, include=['Status'], exclude='bads')
raw.filter(l_freq=0.3, h_freq=None) # filter movement artifacts

events = mne.find_events(raw, stim_channel='Status')

# define baseline for each epoch
tmin, tmax = -0.2, 2
baseline = (None, 0.0)
#reject = {'eeg': 200e-6}
triggers = {'sampleObject': 21, 'sampleScene': 22, 'sampleBlue': 23, 'sampleRed': 24}
epochs = mne.Epochs(raw, events=events, event_id=triggers, tmin=tmin,
                    tmax=tmax, baseline=baseline,
                    picks=('eeg'))
epochs.load_data()
epochs.crop(tmin=0.) # remove baseline period

Extracting EDF parameters from C:\Users\maria\PycharmProjects\EEG_Playground\data\001_D1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1906687  =      0.000 ...   931.000 secs...
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Filter length: 22529 samples (11.000 sec)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B1

<Epochs  |   138 events (all good), 0 - 2 sec, baseline [None, 0], ~552.4 MB, data loaded,
 'sampleBlue': 24
 'sampleObject': 45
 'sampleRed': 24
 'sampleScene': 45>

In [6]:
import numpy as np
from sklearn.svm import SVC

evoked_object = epochs['sampleObject'].get_data()
evoked_scene = epochs['sampleScene'].get_data()

avg_lower = int(np.ceil(0.4 * 2048))
avg_upper = int(np.ceil(2 * 2048))
evoked_object = np.mean(evoked_object[:, :, avg_lower:avg_upper], axis=2)
evoked_scene = np.mean(evoked_scene[:, :, avg_lower:avg_upper], axis=2)
x = np.concatenate([evoked_object, evoked_scene], axis=0)
y = np.concatenate([np.zeros(evoked_object.shape[0]), np.ones(evoked_scene.shape[0])], axis=0)

model = SVC(class_weight='balanced')
model.fit(x, y)
acc = model.score(x, y)
print(acc)

0.5444444444444444


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
# from keras.models import Sequential
# 
# model = Sequential()
# model.add(Input(shape=x.shape[1:]))
# model.add(Conv2D(filters=32, kernel_size=(200, 1), activation='relu'))

import tensorflow as tf
import numpy as np

evoked_object = epochs['sampleObject'].get_data()
evoked_scene = epochs['sampleScene'].get_data()

x = np.concatenate([evoked_object, evoked_scene], axis=0)
y = np.concatenate([np.zeros(evoked_object.shape[0]), np.ones(evoked_scene.shape[0])], axis=0)
x = np.expand_dims(x, 3)

layers = [tf.keras.layers.Conv2D(filters=32, kernel_size=(1, 200), padding='same', input_shape=(128, 4097, 1)),
          tf.keras.layers.Conv2D(filters=32, kernel_size=(1, 200)),
          tf.keras.layers.MaxPool2D(pool_size=(1, 4)),
          tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 200), padding='same'),
          tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 200)),
          tf.keras.layers.MaxPool2D(pool_size=(1, 4)),
          tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 200), padding='same'),
          tf.keras.layers.MaxPool2D(pool_size=(1, 4)),
          tf.keras.layers.Dense(units=64, activation='relu'),
          tf.keras.layers.Dense(units=2, activation='softmax')]

model = tf.keras.Sequential(layers)
model(x, y)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

